In [1]:
from __future__ import division, print_function
import os
import sys
import pandas as pd
from datetime import datetime, timedelta
import hashlib
import logging
import importlib
import numpy as np
import argparse
import shutil
import subprocess as sb

from pyspark.sql import functions as f, types as t
from pyspark.sql import SparkSession, Window

NUM_PARTITIONS = 1024
spark = ( SparkSession
             .builder
             .config("spark.app.name","SmartSaltConfig")
             .config("spark.yarn.queue","root.sp_analytics.spark")
             .config("spark.default.serializer","org.apache.spark.serializer.KryoSerializer")
             .config("spark.speculation","true")
             .config("spark.default.parallelism",str(NUM_PARTITIONS))
             .config("spark.sql.shuffle.partitions",str(NUM_PARTITIONS))
             .config("spark.yarn.executor.memoryOverhead", str(8192))
             .enableHiveSupport()
             .getOrCreate() )

sc = spark.sparkContext

In [2]:
account = 'trivago'
run_date = "2019-05-09"
exp_weights = [1]*20
exp_groups = [1]*20
base_salt = "B"
control = 0
test = 1
period = 14
ntrials = 500
pos = 'All'

In [3]:
current_dir = os.getcwd()
print(current_dir)
current_dir = current_dir[:current_dir.find('notebook')]
mod_file = os.path.join(current_dir, "exp_lib/python/utils/zip_module.py")
sc.addPyFile(mod_file)

/home/lijia/git_tree/lin_jia/meta_experiment_framework/notebook


In [4]:
mod_zipping = importlib.import_module("zip_module")
mod_zipping.zip_and_ship_module(current_dir)
current_dir = os.getcwd()
zipped_file = os.path.join(current_dir,"exp_lib.zip")
sc.addPyFile(zipped_file)

In [5]:
mod = importlib.import_module("exp_lib.account.{0}".format(account))

In [6]:
Metrics = [mod.NitsBookings,mod.NitsProfit]
print("Optimising split on metrics: {}".format([x for x in Metrics]))

Optimising split on metrics: [<class 'exp_lib.account.trivago.NitsBookings'>, <class 'exp_lib.account.trivago.NitsProfit'>]


In [7]:
experiment = mod.account(Metrics,  
                         run_date=run_date,
                         exp_groups=exp_groups,
                         control=control,
                         test=test,
                         exp_weights = exp_weights, 
                         period=period,
                         base_salt=base_salt,
                         ntrials = ntrials, 
                         pos = pos)

In [10]:
salt, pae = experiment.get_smart_salt()

In [15]:
test = [pos]

In [16]:
test == ['All']

True